In [29]:
from json import JSONEncoder, dumps
from datetime import date, datetime
from decimal import Decimal

class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock) or isinstance(obj, Trade):
            result =  obj.as_dict()
            result['object'] = obj.__class__.__name__
            return result
        elif isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%dT%H:%M:%S')
        elif isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, Decimal):
            return str(obj)
        else:
            super().default(obj)

In [30]:
class Stock:
    def __init__(self, symbol, date_, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date_
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol, 
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume)
    
    def __eq__(self,other):
        return isinstance(other,Stock) and self.as_dict()==other.as_dict()
    
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
    def as_dict(self):
        return dict(
            symbol=self.symbol,
            timestamp=self.timestamp,
            order=self.order,
            price=self.price,
            volume=self.volume,
            commission=self.commission)
    
    
        
    def __eq__(self,other):
        return isinstance(other,Trade) and self.as_dict()==other.as_dict()

In [31]:
activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [32]:
encoded = dumps(activity,cls=CustomEncoder,indent=2)
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "order": "buy",
      "price": "338.25",
      "volume": 100,
      "commission": "9.99",
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "volume": 20

In [66]:
def decode_stock(d):
    s = Stock(d['symbol'],
              datetime.strptime(d['date'],'%Y-%m-%d'),
              Decimal(d['open']),
              Decimal(d['high']),
              Decimal(d['low']),
              Decimal(d['close']),
              int(d['volume'])
            )
    return s
                        

In [67]:
d=  decode_stock(  {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    })

In [68]:
vars(d)


{'symbol': 'MSFT',
 'date': datetime.datetime(2018, 11, 22, 0, 0),
 'open': Decimal('103.25'),
 'high': Decimal('103.48'),
 'low': Decimal('103.07'),
 'close': Decimal('103.11'),
 'volume': 4493689}

In [69]:
def decode_trade(d):
    s = Trade(d['symbol'],
              datetime.strptime(d['timestamp'],'%Y-%m-%dT%H:%M:%S'),
              d['order'],
              Decimal(d['price']),
              int(d['volume']),
              Decimal(d['commission'])
            )
    return s
                

In [70]:
t= decode_trade ({
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "volume": 20,
      "commission": "9.99",
      "object": "Trade"
    })

In [71]:
type(t)

__main__.Trade

In [72]:
vars(t)

{'symbol': 'AAPL',
 'timestamp': datetime.datetime(2018, 11, 22, 10, 30, 5),
 'order': 'sell',
 'price': Decimal('177.01'),
 'commission': Decimal('9.99'),
 'volume': 20}

In [73]:
def decode_financials(d):
    object_type = d.get('object',None)
    if object_type =='Stock':
        return decode_stock(d)
    elif object_type =='Trade':
        return decode_trade(d)
    return d

In [74]:
decode_financials({
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "volume": 20,
      "commission": "9.99",
      "object": "Trade"
    })

In [75]:
decode_financials(  {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    })

In [76]:
decode_financials({'a':1})

{'a': 1}

In [78]:
from json import JSONDecoder, loads

In [82]:
class CustomDecoder(JSONDecoder):
    def decode(self,arg):
        data = loads(arg)
        return self.parse_financials(data)
    
    def parse_financials(self,obj):
        if isinstance(obj,dict):
            obj = decode_financials(obj)
            if isinstance(obj,dict):
                for key,value in obj.items():
                    obj[key] = self.parse_financials(value)
        elif isinstance(obj,list):
            for index, item in enumerate(obj):
                obj[index] = self.parse_financials(item)
        return obj

In [83]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "order": "buy",
      "price": "338.25",
      "volume": 100,
      "commission": "9.99",
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "volume": 20

In [84]:
decoded = loads(encoded,cls=CustomDecoder)

In [85]:
decoded

{'quotes': [<__main__.Stock at 0x110fb0128>,
 'trades': [<__main__.Trade at 0x110fb0d30>, <__main__.Trade at 0x110fb0b00>]}

In [86]:
decoded == activity

False

In [90]:
class Stock:
    def __init__(self, symbol, date_, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date_
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol, 
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume)
    
    def __eq__(self,other):
        return isinstance(other,Stock) and self.as_dict()==other.as_dict()
    
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
    def as_dict(self):
        return dict(
            symbol=self.symbol,
            timestamp=self.timestamp,
            order=self.order,
            price=self.price,
            volume=self.volume,
            commission=self.commission)
    
    
        
    def __eq__(self,other):
        return isinstance(other,Trade) and self.as_dict()==other.as_dict()

In [91]:
activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [92]:
encoded = dumps(activity,cls=CustomEncoder,indent = 2)
decoded = loads(encoded,cls=CustomDecoder)
activity == decoded

False